In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

**Replace hajmaj24 with your sudo password in all relevant cells**

In [2]:
!echo hajmaj24 | sudo -S pip install gitpython

Password:Requirement already satisfied: gitpython in /Users/ayushimohan/opt/anaconda3/lib/python3.7/site-packages (3.1.3)


In [3]:
!echo hajmaj24 | sudo -S pip install opencv-python

Password:Requirement already satisfied: opencv-python in /Users/ayushimohan/opt/anaconda3/lib/python3.7/site-packages (4.2.0.34)


In [4]:
import os
import git

In [5]:
os.getcwd()

'/Users/ayushimohan/Documents/GitHub/PESU-ParkIt'

In [6]:
if not os.path.exists("Mask_RCNN"):
    print("Cloning M-RCNN repository...")
    git.Git("./").clone("https://github.com/matterport/Mask_RCNN.git")

os.chdir("Mask_RCNN")


!echo hajmaj24 | sudo -S pip install -r requirements.txt
%run setup.py install

Password:Requirement already satisfied: numpy in /Users/ayushimohan/opt/anaconda3/lib/python3.7/site-packages (from -r requirements.txt (line 1)) (1.18.1)


running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.9-x86_64/egg
running install_lib
running build_py
creating build/bdist.macosx-10.9-x86_64/egg
creating build/bdist.macosx-10.9-x86_64/egg/mrcnn
copying build/lib/mrcnn/config.py -> build/bdist.macosx-10.9-x86_64/egg/mrcnn
copying build/lib/mrcnn/parallel_model.py -> build/bdist.macosx-10.9-x86_64/egg/mrcnn
copying build/lib/mrcnn/__init__.py -> build/bdist.macosx-10.9-x86_64/egg/mrcnn
copying build/lib/mrcnn/visualize.py -> build/bdist.macosx-10.9-x86_64/egg/mrcnn
copying build/lib/mrcnn/model.py -> build/bdist.macosx-10.9-x86_64/egg/mrcnn
copying build/lib/mrcnn/utils.py -> buil

zip_safe flag not set; analyzing archive contents...


creating 'dist/mask_rcnn-2.1-py3.7.egg' and adding 'build/bdist.macosx-10.9-x86_64/egg' to it
removing 'build/bdist.macosx-10.9-x86_64/egg' (and everything under it)
Processing mask_rcnn-2.1-py3.7.egg
Removing /Users/ayushimohan/opt/anaconda3/lib/python3.7/site-packages/mask_rcnn-2.1-py3.7.egg
Copying mask_rcnn-2.1-py3.7.egg to /Users/ayushimohan/opt/anaconda3/lib/python3.7/site-packages
mask-rcnn 2.1 is already the active version in easy-install.pth

Installed /Users/ayushimohan/opt/anaconda3/lib/python3.7/site-packages/mask_rcnn-2.1-py3.7.egg
Processing dependencies for mask-rcnn==2.1
Finished processing dependencies for mask-rcnn==2.1


In [7]:
!echo hajmaj24 | sudo -S pip install tensorflow==1.13.1
!echo hajmaj24 | sudo -S pip install keras==2.1.0

Password:Requirement already satisfied: tensorflow==1.13.1 in /Users/ayushimohan/opt/anaconda3/lib/python3.7/site-packages (1.13.1)
Password:Requirement already satisfied: keras==2.1.0 in /Users/ayushimohan/opt/anaconda3/lib/python3.7/site-packages (2.1.0)


In [8]:
import tensorflow as tf
tf.__version__

'1.13.1'

In [9]:
import keras as k
k.__version__

Using TensorFlow backend.


'2.1.0'

In [10]:
import numpy as np
import cv2
import Mask_RCNN.mrcnn.config
import Mask_RCNN.mrcnn.utils
from Mask_RCNN.mrcnn.model import MaskRCNN
from pathlib import Path
import pickle
import argparse

from shapely.geometry import box
from shapely.geometry import Polygon as shapely_poly
import io
import base64
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)


class Config(Mask_RCNN.mrcnn.config.Config):
    NAME = "model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 81

config = Config()
config.display()

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

print(COCO_MODEL_PATH)
if not os.path.exists(COCO_MODEL_PATH):
    Mask_RCNN.mrcnn.utils.download_trained_weights(COCO_MODEL_PATH)

model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=Config())
model.load_weights(COCO_MODEL_PATH, by_name=True)

def get_cars(boxes, class_ids):
    cars = []
    for i, box in enumerate(boxes):
        if class_ids[i] in [3, 8, 6]:
            cars.append(box)
    return np.array(cars)

def compute_overlaps(parked_car_boxes, car_boxes):
    new_car_boxes = []
    for box in car_boxes:
        y1 = box[0]
        x1 = box[1]
        y2 = box[2]
        x2 = box[3]
        
        p1 = (x1, y1)
        p2 = (x2, y1)
        p3 = (x2, y2)
        p4 = (x1, y2)
        new_car_boxes.append([p1, p2, p3, p4])
    
    overlaps = np.zeros((len(parked_car_boxes), len(new_car_boxes)))
    for i in range(len(parked_car_boxes)):
        for j in range(car_boxes.shape[0]):
            pol1_xy = parked_car_boxes[i]
            pol2_xy = new_car_boxes[j]
            polygon1_shape = shapely_poly(pol1_xy)
            polygon2_shape = shapely_poly(pol2_xy)

            polygon_intersection = polygon1_shape.intersection(polygon2_shape).area
            polygon_union = polygon1_shape.union(polygon2_shape).area
            IOU = polygon_intersection / polygon_union
            overlaps[i][j] = IOU
    return overlaps



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [11]:
os.chdir("..")
os.getcwd()

'/Users/ayushimohan/Documents/GitHub/PESU-ParkIt'

In [12]:
!echo hajmaj24 | sudo -S pip install opencv-contrib-python

Password:Requirement already satisfied: opencv-contrib-python in /Users/ayushimohan/opt/anaconda3/lib/python3.7/site-packages (4.2.0.34)


In [13]:
smh = open("smh.txt", "r")
id = smh.read()
print(id)

5


In [ ]:
if __name__ == "__main__":
    
    regions = "regions-1.p"
    with open(regions, 'rb') as f:
        parked_car_boxes = pickle.load(f)

    VIDEO_SOURCE = "parking_vid.mp4"
    alpha = 0.6
    video_capture = cv2.VideoCapture(VIDEO_SOURCE)
    video_FourCC    = cv2.VideoWriter_fourcc('M','J','P','G')
    video_fps       = video_capture.get(cv2.CAP_PROP_FPS)
    video_size      = (int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    out = cv2.VideoWriter("out.avi", video_FourCC, video_fps, video_size)
    id_list = []
    while video_capture.isOpened():
        success, frame = video_capture.read()
        overlay = frame.copy()
        if not success:
            print("Missing")
            break

        rgb_image = frame[:, :, ::-1]
        results = model.detect([rgb_image], verbose=0)
        
        
        cars = get_cars(results[0]['rois'], results[0]['class_ids'])
        overlaps = compute_overlaps(parked_car_boxes, cars)

        for parking_area, overlap_areas in zip(parked_car_boxes, overlaps):
            count = 1
            max_IoU_overlap = np.max(overlap_areas)
            if max_IoU_overlap < 0.15:
                cv2.fillPoly(overlay, [np.array(parking_area)], (0, 0, 255))
                free_space = True
                id_list.append(count)
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
        
        '''count = 0
        while count != id:
             frame = cv2.putText(np.array(frame), str(count+1), (int(parked_car_boxes[0][0,:1]), int(parked_car_boxes[0][0,1:])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
             count += 1'''

        cv2.imshow('output', frame)
        out.write(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    out.release()
    cv2.destroyAllWindows()
    print("output saved as out.avi")
    print(id_list)